### 저작자 김민규 김규림

In [3]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install openai==1.55.3 httpx==0.27.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [6]:
from crewai import Agent, Task, Crew

In [1]:
import os
import openai
import json

# JSON 파일 로드
with open("API_KEYS.json", "r") as file:
    api_keys = json.load(file)

# OPENAI API KEY 설정
os.environ['OPENAI_API_KEY']=api_keys["OPENAI_API_KEY"]
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'


Agent(에이전트) 생성하기

Agent를 정의하고, role(역할), goal(목표), backstory(배경 설명)를 제공합니다.

LLM(대규모 언어 모델)은 롤플레잉을 할 때 더 나은 성능을 보이는 것으로 확인되었습니다.

### Agent: 프로듀서(PRODUCER)

**참고**: _여러 문자열_ 사용의 이점:
```Python
varname = "텍스트의 첫 번째 줄"
          "텍스트의 두 번째 줄"
```

_삼중 따옴표 독스트링_ 사용과 비교했을 때:
```Python
varname = """텍스트의 첫 번째 줄
             텍스트의 두 번째 줄
          """
```
여러 문자열을 사용하면 공백과 줄바꿈 문자가 추가되는 것을 피할 수 있어, LLM에 전달할 때 더 나은 형식을 유지할 수 있습니다.

In [54]:
planner = Agent(
    role="프로듀서",
    goal="{topic}에 대한 흥미롭고 자극적인 콘텐츠 기획",
    backstory="당신은 {topic}에 대한 예능 프로그램을 "
              "기획하는 일을 하고 있습니다. "
              "시청자들이 흥미를 느끼고"
              "도파민을 폭발시킬 수 있는"
              "재미있고 자극적인 사실에 기반한"
              "티비 프로그램을 기획합니다"
              "또한 출연할 스캔들 주인공 2명과 티비 프로그램을 진행할 연예인 1명을 정하고 섭외합니다."
              "이때, 스캔들 주인공 2명은 실존 인물로 "
              "남녀 구성으로 남자는 30대 스타 배우, 여성은 20대 유명 그룹 아이돌입니다."
              "당신의 작업은 작가가 "
              "이 주제로 대본을 쓸 수 있는 토대를 만들어 주는 겁니다."
              "결과물은 한국어로 생성될 예정입니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: 작가(Writer)

In [55]:
writer = Agent(
    role="작가",
    goal="최근 화제가 된 {topic}에 대해"
         "강렬하고 자극적인 대본을 작성",
    backstory="당신은 {topic}에 대한 최근 이슈를 종합해서 "
              "시청자들이 관심을 가질만한 대본을 작성합니다. "
              "프로듀서가 제공한 개요와 출연진을 바탕으로"
              "주제와 관련된 이슈를 소개하고 "
              "그에 대한 스캔들 당사자인 출연진들의 대사를 작성합니다."
              "프로듀서가 제시한 "
              "주요 목표와 방향을 따릅니다. "
              "또한 출연진들이 자신의 입장을 대변하는 "
              "솔직하고 자극적인 견해를 작성하고"
              "이를 이끌어내는 "
              "진행자의 멘트를 첨가합니다. "
              "해당 주제가 진실이라는 가정 하에 대본에서 출연진들의 진술과 "
              "주요 진행자의 견해를 화기애애한 분위기 속에서 펼쳐냅니다."
              "결과물은 한국어로 생성될 예정입니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: 음향감독(Sound Director)

In [56]:
editor = Agent(
    role="음향감독",
    goal="주어진 상황과 대사에 맞는 음향을 재생",
    backstory="당신은 작가로부터 "
              "받은 대본을 기반으로 상황에 맞는 음향을 재생하는 음향감독입니다. "
              "대본을 검토하여 주어진 상황과 대사에 맞는 "
              "가요를 배경음악으로 선정하고 각 대사의 분위기에 맞는 효과음을 대본 속에 추가합니다. "
              "예를 들어, 주제와 관련된 인물이 가수라면 "
              "해당 인물이 진술할때 인물의 노래를 재생합니다."
              "가능한 상황과 연관되고 "
              "시청자들의 가슴을 뛰게 할 음향을 준비하는 것이 당신의 역할입니다."
              "결과물은 한국어로 생성될 예정입니다.",
    allow_delegation=False,
    verbose=True
)

## Task(작업) 생성하기

- Task를 정의하고, `description`(설명), `expected_output`(예상 결과물), `agent`(수행 에이전트)를 제공합니다.


### Task: Plan

In [57]:
plan = Task(
    description=(
        "1. {topic}에 대한 최신 트렌드, 주요 인물, "
            "그리고 주목할 만한 뉴스를 우선순위화하기.\n"
        "2. 세간의 관심사와 선호하는 연예인을 "
            "고려하여 프로그램을 기획하기.\n"
        "3. 주제의 개요와 출연진들의 인터뷰를 포함한 "
            "상세한 프로그램 개요 개발하기.\n"
        "4. 주제와 관련된 자료의 출처와 출연진 명단 포함하기."
    ),
    expected_output="개요, 관심사, 선호되는 연예인 "
        "주제 관련 자료, 출연진 명단을 포함한 "
        "포괄적인 프로그램 기획 문서."
        "결과물은 한국어로 생성될 예정입니다.",
    agent=planner,
)

### Task: Write

In [58]:
write = Task(
    description=(
        "1. 프로그램 기획을 활용하여 {topic}에 대한 "
            "매력적인 대본 작성하기.\n"
        "2. 시청자들의 이목을 끌 자극적이고 흥미로운 키워드 포함하기.\n"
        "3. 섹션/부제목을 흥미롭게 적절한 "
            "이름으로 지정하기.\n"
        "4. 매력적인 서론, 출연진들의 진실성 있는 발언을 포함한 본문, "
            "요약하는 결론으로 대본을 구성하기.\n"
        "5. 출연진의 실명을 그대로 노출시키고 "
            "즐거운 프로그램 분위기가 형성되도록 작성하기.\n"
    ),
    expected_output="각 섹션이 2~3개의 문단으로 구성된, "
        "송출 준비가 완료된 토크쇼 형식의 "
        "재미있게 작성된 대본."
        "결과물은 한국어로 생성될 예정입니다.",
    agent=writer,
)

### Task: Edit

In [59]:
edit = Task(
    description=("주어진 대본의 분위기를 검토하고 "
                 "적잘한 음향을 삽입합니다."),
    expected_output="각 섹션이 2~3개의 문단으로 구성된, "
                    "송출 준비가 완료된 토크쇼 형식의 "
                    "재미있게 작성된 대본."
                    "결과물은 한국어로 생성될 예정입니다.",
    agent=editor
)

## Crew(팀) 생성하기

- Agent들로 구성된 팀을 생성합니다
- 해당 Agent들이 수행할 작업들을 전달합니다.
    - **참고**: *이 간단한 예시에서는* 작업들이 순차적으로 수행됩니다(즉, 서로 의존적임). 따라서 목록에서의 작업 _순서_가 _중요_합니다.
- `verbose=2`를 설정하면 실행의 모든 로그를 확인할 수 있습니다.


이 설정에서:
1. `planner`가 먼저 콘텐츠를 기획합니다
2. `writer`가 기획된 내용을 바탕으로 글을 작성합니다
3. `editor`가 최종적으로 작성된 글을 검토하고 편집합니다

In [60]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2025-02-05 16:21:02,947 - 140363700756480 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

In [61]:
result = crew.kickoff(inputs={"topic": "정치를 제외한 연예계에서 발생한 탑스타들의 스캔들"})

 [DEBUG]: == Working Agent: 프로듀서
 [INFO]: == Starting Task: 1. 정치를 제외한 연예계에서 발생한 탑스타들의 스캔들에 대한 최신 트렌드, 주요 인물, 그리고 주목할 만한 뉴스를 우선순위화하기.
2. 세간의 관심사와 선호하는 연예인을 고려하여 프로그램을 기획하기.
3. 주제의 개요와 출연진들의 인터뷰를 포함한 상세한 프로그램 개요 개발하기.
4. 주제와 관련된 자료의 출처와 출연진 명단 포함하기.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 

프로그램 기획서: "스타 스캔들: 그 후의 이야기"

1. 개요:
"스타 스캔들: 그 후의 이야기"는 연예계에서 가장 뜨거운 스캔들에 휘말린 스타들을 초대하여 그들의 이야기를 듣고, 그들이 스캔들을 어떻게 극복했는지, 그리고 그 후의 삶은 어떻게 변했는지를 조명하는 예능 프로그램입니다. 시청자들은 그들의 솔직하고 진솔한 이야기를 통해 연예계의 이면을 엿볼 수 있습니다.

2. 관심사:
- 대중들이 관심을 가질 만한 최근 스캔들
- 스캔들의 영향력과 파급력
- 스캔들을 극복하는 과정과 그 후의 변화

3. 선호되는 연예인 주제 관련 자료:
- 최근 몇 년간 큰 화제를 모았던 스캔들 리스트 및 관련 기사
- 스캔들로 인한 대중의 반응과 여론 분석 자료
- 스캔들 이후 스타들의 활동 및 인터뷰 자료

4. 출연진 명단:
- 남자 스타: 김수현 (30대 스타 배우)
최근 작품 및 인터뷰에서 보여준 솔직한 매력과 대중적 인기를 바탕으로 섭외. 스캔들 경험이 풍부하여, 그에 대한 솔직한 이야기를 기대할 수 있습니다.

- 여자 스타: 리사 (20대 유명 그룹 아이돌, 블랙핑크)
국내외에서 큰 인기를 얻고 있는 아이돌로, 최근 개인 활동과 그룹 활동에서의 이슈를 바탕으로 섭외. 그녀의 글로벌 팬덤과 영향력을 고려하여 프로그램의 주목도를 높일 수 있습니다.

- 진행자: 유재

In [62]:
from IPython.display import Markdown
Markdown(result)

---

**[오프닝]**

**유재석:**  
(배경 음악: 경쾌한 인트로 음악)  
안녕하세요, 여러분! "스타 스캔들: 그 후의 이야기"에 오신 걸 환영합니다. 오늘은 여러분이 궁금해하던 스타들의 솔직한 이야기를 들어보는 시간을 마련했습니다. 그들이 어떤 스캔들을 겪었고, 어떻게 극복했는지를 직접 들어볼 수 있는 기회죠. 오늘 함께할 특별한 게스트, 김수현 씨와 리사 씨를 소개합니다!

(청중의 박수 소리)  

**김수현:**  
안녕하세요, 김수현입니다. 이렇게 솔직한 이야기의 장이 마련되니 저도 기대가 됩니다.

**리사:**  
안녕하세요! 블랙핑크의 리사입니다. 저도 많이 기대하고 왔어요!

(배경 음악: 블랙핑크의 "How You Like That"의 짧은 인트로)

---

**[인터뷰 세션 1: 스캔들 경험과 당시 심경]**

**유재석:**  
(조용한 긴장감 있는 음악)  
수현 씨, 먼저 시작해볼까요? 몇 년 전 대중의 큰 관심을 끌었던 그 사건이 있었죠. 당시 상황은 어땠나요?

**김수현:**  
네, 그때는 정말 힘들었습니다. 하루아침에 모든 게 바뀌더라고요. 대중의 시선이 무거웠지만, 오히려 그 경험이 저를 더 단단하게 만든 것 같아요.

(배경 음악: 감성적인 피아노 연주)

**유재석:**  
리사 씨는 어떠셨어요? 글로벌 팬들이 많은 만큼, 더 큰 부담이 있었을 것 같은데요.

**리사:**  
맞아요, 팬들의 기대와 실망이 동시에 느껴지더라고요. 하지만 멤버들과 팬들의 응원이 있었기에 잘 이겨낼 수 있었습니다.

(배경 음악: 블랙핑크의 "Stay"의 부드러운 멜로디)

---

**[인터뷰 세션 2: 스캔들 이후의 일상과 커리어 변화]**

**유재석:**  
스캔들 이후에 두 분의 삶도 많이 변했을 것 같은데요. 수현 씨는 어떤 변화를 느꼈나요?

**김수현:**  
스캔들 이후로 작품에 더 몰입하게 되었어요. 연기에 대한 진정성을 더 깊이 고민하게 되었고, 그 덕분에 더 좋은 작품으로 팬들을 만날 수 있었던 것 같습니다.

(배경 음악: 감동적인 오케스트라 음악)

**리사:**  
저는 솔로 활동을 시작하면서 저 자신에게 더 집중하게 되었어요. 저를 믿어주는 팬들에게 보답하기 위해 더욱 열심히 노력했습니다.

(배경 음악: 리사의 "LALISA"의 파워풀한 인트로)

---

**[인터뷰 세션 3: 대중의 반응과 극복 과정]**

**유재석:**  
두 분 모두 대중의 반응을 직접적으로 느끼셨을 텐데요. 이를 극복하는 데 어떤 노력이 필요했나요?

**김수현:**  
대중의 비판을 수용하면서도 제 자신을 잃지 않으려 했어요. 중요한 것은 스스로의 가치와 신념을 잃지 않는 것이라고 생각합니다.

(배경 음악: 희망적인 스트링 연주)

**리사:**  
저는 긍정적인 사고방식을 유지하려고 노력했어요. 시간이 걸리더라도 스스로를 믿고 꾸준히 나아가면 결국 좋은 결과가 따라온다고 생각합니다.

(배경 음악: 밝은 피아노 선율)

---

**[마무리]**

**유재석:**  
오늘 두 분의 솔직한 이야기 정말 감사합니다. 마지막으로 시청자들에게 전하고 싶은 메시지가 있을까요?

**김수현:**  
힘든 시기를 겪고 있는 분들께, 그 시간을 통해 더 나은 자신을 발견할 수 있다고 말씀드리고 싶어요.

**리사:**  
저는 언제나 여러분을 응원합니다. 여러분도 포기하지 말고 꿈을 향해 나아가세요!

(배경 음악: 감동적인 피날레 음악)

**유재석:**  
오늘 함께해 주신 김수현 씨, 리사 씨 정말 감사합니다. "스타 스캔들: 그 후의 이야기"는 여기서 마무리하겠습니다. 다음 시간에도 솔직하고 흥미로운 이야기로 찾아뵙겠습니다. 감사합니다!

(배경 음악: 프로그램 테마곡 엔딩)

---

- Display the results of your execution as markdown in the notebook.